In [9]:
def ScoreRunnerFinal(nthround,P1State,P2State):
    # The length of the code is 5, 3. 
    # There is a plot called output.png that shows that this combination is the best. (In the github link)
    # See the CodeLength.ipynb for more details. (Also in the github link)
    # For the choice of the code, refer to Tristan's message & code

    # Supporters play the code then cooperate from then on (including while runner is playing their code)
    PaceMakerMoves = ['d', 'd', 'c', 'c', 'c'] + ['c']*(200 - 5)
    # Runners play the default strategy for 5 turns then play the runner code then defect from then on
    RunnerMoves = ['c', 'd', 'd', 'd', 'c', 'c', 'd', 'c'] + ['d']*(200 - 8)

    # Is the opponent a ScorePaceMaker?
    # If so, play the moves we agreed on
    if P2State[:max(5, nthround)] == PaceMakerMoves[:max(5, nthround)]:
        P1State[nthround] = RunnerMoves[nthround]
        return P1State

    # A block of 10 'd's at the end 
    if 190 <= nthround:
        P1State[nthround] = "d"
        return P1State

    # Stepped GRIM Trigger
    # Trigger with 4 defections in the last 50 rounds
    # Trigger with 3 defections in the last 35 rounds
    # Trigger with 2 defections in the last 20 rounds
    # Trigger with 1 defection in the last 15 rounds
    TriggerRounds = [15, 20, 35, 50]
    for index, trigger in enumerate(TriggerRounds):
        if any(P2State[pos:pos + index + 1] == ['d']*(index+1) for pos in range(199 - trigger, 200)):
            P1State[nthround] = "d"
            return P1State

    # Otherwise, play the modified version of softMAJ + RATIO
    # It adds 'forgiveness' and 'forgetfulness' to softMAJ (explained below)

    # Forgiveness bit (The essense of Alex's RATIO strategy)
    if nthround == 27: # at round 27, (chosen at random)
        # if the opponent defected only once more than 1/2 of the time and 
        # if we have non-negative score so we can afford to be nice
        nc2 = 2*P2State.count("c")
        if nc2 >= P1State.count("c") and nc2 >= nthround - 1: 
            # then forgive them and play 'c' in the hope that we can cooperate from then on
            P1State[nthround] = "c"
            return P1State
    
    # Forgetfulness bit
    # An arbitrary weighting curve that somehow works fine
    def forgetfulness(round): 
        if round < 9:
            return 1
        else:
            return 0.5**(round-9) *0.3 + 0.7
    
    # Rather than using the number of defections (nd) and the number of cooperations (nc) directly,
    # we use the weighted sum of the number of defections and cooperations (weighted by forgetfulness function above)
    # Recent rounds are weighted more heavily than older rounds
    flip = P2State[:nthround][::-1]
    nd = sum([forgetfulness(i) for i, x in enumerate(flip) if x == "d"])
    nc = sum([forgetfulness(i) for i, x in enumerate(flip) if x == "c"])

    # Normal softMAJ stuff (just made short)
    P1State[nthround] = "c" if nd <= nc else "d"
    return P1State
